# What is a view?
The main logic of the interaction of applications with a user is described in views. Function-based views or Class-based views are responsible for how requests are processed and what answers 

# Preparations for simple views


In [ ]:
django-admin startproject blog
cd blog
python3 manage.py startapp blogsite

For example, we can write these views in ```views.py``` file:

In [ ]:
from django.http import HttpResponse
  
def index(request):
    return HttpResponse("Welcome to blog of Django learners!")

def list_articles(request):
    return HttpResponse("List of articles")

def contributors(request):
    return HttpResponse("Contributors of our community")

To make this HttpResponse work, add URL patterns to urls.py in the blog directory. The urlpatterns variable determines a set of request comparisons and the view functions to proceed with them.

As we have the ```urls.py``` file in the folder of the project, we'll connect it with the application's ```views.py``` file as follows with the include function:

In [ ]:
from django.contrib import admin
from django.urls import path, include

urlpatterns = [
    path('admin/', admin.site.urls),
    path('blogsite/', include('blogsite.urls')),
]

The next step is to create a ```urls.py``` file inside the application blogsite manually, and that's where our URL addresses are to be placed:

In [ ]:
from django.urls import path
from . import views

urlpatterns = [
    path('', views.index, name='index'),
    path('list_of_articles/', views.list_articles, name='list'),
    path('contributors/', views.contributors),
    ]

That's all! Now, let's run the following command:

``` python3 manage.py runserver```


Nice! A bit too old-fashioned and straightforward for our goals. Of course, we can place some HTML or even big pieces of code into such view functions, but that's not useful and hard to support in case of vast projects. So let's go further and make something more exciting and informative.

By the way, we haven't yet tried to add kwargs to our view function, which is also an available option. For example, we could pass some changing data to the page of our blog:

In [ ]:
# in the views.py file:
def contributors(request, age, name, experience):
    return HttpResponse(f"""
            <h2>Contributor of our community:</h2>
            <p>Name: {name}</p>
            <p>Age: {age}</p>
            <p>Experience and some info you'd like to share: {experience}</p>
    """)

In [ ]:
# in the urls.py file:
urlpatterns = [
    path('contributors/', views.contributors, kwargs={"name": "Robert May", "age": 38, "experience": 
    "Two years in startup, graduated from MIT in 2020"}),
    ]

Not much can be done with this functionality, but such mechanisms can be used as a base for our coding fantasy.

- Since our blog needs a database to store the information our future community members will share, let's create models in models.py file for articles and contributors (or authors, if you like):



In [ ]:
from django.db import models
from django.contrib.auth.models import User
from datetime import date
from django.urls import reverse


class Article(models.Model):
    title = models.CharField(max_length=250, help_text='Enter a title for a new article')
    text = models.CharField(max_length=1000, help_text='Place your story here')
    author = models.ForeignKey('Contributor', on_delete=models.SET_NULL, null=True)
    date = models.DateTimeField(null=True, blank=True)

    def __str__(self):
        return self.title


class Contributor(models.Model):
    name = models.CharField(max_length=250, help_text='Enter first name of the contributor')
    surname = models.CharField(max_length=500, help_text='Enter surname of the contributor')
    about = models.CharField(max_length=2000, help_text='Tell us about yourself, your experience and current work')
    email = models.EmailField(null=True, blank=True)

    class Meta:
        ordering = ['surname', 'name']

    def __str__(self):
        return f'{self.surname}, {self.name}'

Don't forget to register your models in the ```admin.py``` file:



In [ ]:
from django.contrib import admin
from .models import Article, Contributor

admin.site.register(Article)
admin.site.register(Contributor)

Also, remember to migrate changes in the project:

```
python3 manage.py makemigrations
python3 manage.py migrate
```

We will also need a template to represent the results of data processing. Don't forget that we should add the templates/blogsite folder to the tree of the application manually:

We'll start with the ```index.html``` page, which will be the list of articles in the blog. For example, the template may look like this:

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Greetings page</title>
</head>
<body>
  <h1>Hello, {{ greetings_to }} !</h1>
  <h1>Unread articles for today: {{num_articles }}</h1>
  {% for article in articles %}
    <h4>{{ article.title }}</h4>
    <h4>{{ article.author }}</h4>
    <p>{{ article.date }}</p>
    <p>{{ article.text }}</p>
    <be>
  {% endfor %}
</body>
</html>

One more preparation step, and we'll be ready to create a view function at last. Let's modify urlpatterns in the urls.py file:

In [ ]:
urlpatterns = [
    path('', views.index, name='articles_list'),
    ]

## Create view functions

We can now create a view function for the index page of our blog:



In [ ]:
def index(request):
    greetings_to = 'Anonymous'
    num_articles = Article.objects.all().count()
    articles = Article.objects.all
    return render(request, 'blogsite/index.html', {'num_articles': num_articles, 'greetings_to': greetings_to, 'articles': articles})

Here we define the variables that we use in the HTML template, get data from the database (which is called a QuerySet, a list of objects of a model) and pass them as a dictionary to the render function.

It's nice to see our work's results already, right? But to make our view function look prettier, let's form a special dictionary named context which is more convenient to work with:

In [ ]:
# views.py file
def index(request):
    num_articles = Article.objects.all().count()
    articles = Article.objects.all
    context = {
        'greetings_to': 'Anonymous',
        'num_articles': num_articles,
        'articles': articles,
    }
    return render(request, 'blogsite/index.html', context=context)